# Queries for the search application

In [1]:
import pandas as pd
import json
import pymongo
import io
import pprint
import itertools

/usr/local/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/local/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [4]:
#Connect to the mongoDB database
client = pymongo.MongoClient()
db = client["tweet_database"]
tweets = db.tweets_collection

In [5]:
#Document(row) sample from the mongoDB database 
myquery = {}
mydoc = list(tweets.find(myquery))
mydoc[0]

{'_id': ObjectId('5ea77ccfc9283bc56a26dae1'),
 'id_str': '1254934763736928256',
 'user_name': 'Jay@pandiyan',
 'user_id': '1188788321532559361',
 'content': 'The U.S. economy could contract 20%-30 in Q2, White House senior economic advisor Kevin Hassett said this morning. “For second-quarter GDP, it’s going to be the biggest negative number that we’ve seen since the Great Depression.” https://t.co/xMTKVRbmU1 https://t.co/cF9nyAEh2g',
 'created_at': '2020-04-28 00:45:37',
 'location': 'Tamil Nadu, India',
 'hashtags': [],
 'mentions': ['20402945'],
 'in_reply_to_user_id': None,
 'in_reply_to_status_id': None,
 'retweetedFrom': '20402945',
 'media': [],
 'retweet': True,
 'FavCount': -1,
 'source': 'Twitter for Android',
 'retweet_count': 9}

## Lets learn about our data

In [8]:
## Original vs retweet count
tweet_type = {'original': 0, 'retweets': 0}

for tweet in tweets.find():
    if tweet['retweet'] == True:
        tweet_type['original'] += 1
    else:
        tweet_type['retweets'] += 1
tweet_type

{'original': 12055, 'retweets': 3514}

In [10]:
## Type of media in our dataset
type_count = {"text": 0, "Only_image":0, "both":0}
for tweet in tweets.find():
    md = tweet["media"]
    if len(md) != 0:
        if tweet["content"] == "":
            type_count["image"] += 1 # Only Image
        else:
            type_count["both"] += 1 # Image and text
    else:
        type_count["text"] += 1 # Only text

type_count

{'text': 14402, 'Only_image': 0, 'both': 1167}

In [12]:
## Number of distint user in database
distinct_users = set()

for tweet in tweets.find():
    distinct_users.add(tweet['user_id'])

print("{} distinct user out of {} total user".format(len(distinct_users), tweets.count()))

14155 distinct user out of 15569 total user


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  


In [16]:
## Tweet with  maximum retweet count
max_retweet_count = 0
for tweet in tweets.find():
    if tweet['retweet_count']>max_retweet_count:
        max_retweet_count = tweet['retweet_count']
        tweet_id = tweet['id_str']
print(max_retweet_count)

myquery = {'id_str': tweet_id}
twts = list(tweets.find(myquery))
twts[0]

12676


{'_id': ObjectId('5ea77dc7c9283bc56a270ebc'),
 'id_str': '1254112304754556929',
 'user_name': 'J',
 'user_id': '785827332',
 'content': "3 companies with ties to the Trump admin. received millions under the PPP program. Another got a loan from a bank that once employed its board's chair. https://t.co/GcgqwGPe43",
 'created_at': '2020-04-25 18:17:28',
 'location': 'The World',
 'hashtags': [],
 'mentions': ['14173315'],
 'in_reply_to_user_id': None,
 'in_reply_to_status_id': None,
 'retweetedFrom': '14173315',
 'media': [],
 'retweet': True,
 'FavCount': -1,
 'source': 'Twitter for iPhone',
 'retweet_count': 12676}

In [17]:
## Tweet with  maximum favorite count
max_FavCount = 0
for tweet in tweets.find():
    if tweet['FavCount']>max_FavCount:
        max_FavCount = tweet['FavCount']
        tweet_id = tweet['id_str']
print(max_FavCount)

myquery = {'id_str': tweet_id}
twts = list(tweets.find(myquery))
twts[0]

4708


{'_id': ObjectId('5ea77daec9283bc56a2709fb'),
 'id_str': '1254168393638449153',
 'user_name': 'POLITICO',
 'user_id': '9300262',
 'content': 'Dr. Anthony Fauci says the U.S. should at least double coronavirus testing in the coming weeks before easing into reopening the economy https://t.co/9LrRsvl1qX',
 'created_at': '2020-04-25 22:00:21',
 'location': 'Washington, D.C.',
 'hashtags': [],
 'mentions': [],
 'in_reply_to_user_id': None,
 'in_reply_to_status_id': None,
 'retweetedFrom': -1,
 'media': [],
 'retweet': False,
 'FavCount': 4708,
 'source': 'Hootsuite Inc.',
 'retweet_count': 1752}

## Search by word

In [4]:
#Find number of tweets with word
myquery = {"content":{"$regex":"covid19","$options" :'i'}}
tweets.count_documents(myquery)

413

In [5]:
#Tweets
twts = list(tweets.find(myquery))
twts[0]

{'_id': ObjectId('5ea66a14fb8f5ff7fd4712a9'),
 'id_str': '1254636770051928071',
 'user_name': 'Pamela Moore #WWG1WGA #DIGITALSOLDIERS',
 'user_id': '1188695886781698050',
 'content': 'Met virtually with our terrific @AmChamPH Board on how U.S. firms are supporting the Philippine economy, workforce, and #COVID19 response. We’re all in this together, and great to \u200bsee U.S. firms contributing in so many ways. #FriendsPartnersAllies #goodbUSinessPH',
 'created_at': '2020-04-27 05:01:30',
 'location': 'Broken Arrow, OK',
 'hashtags': [],
 'mentions': ['441540997', '4745918652'],
 'in_reply_to_user_id': None,
 'in_reply_to_status_id': None,
 'retweetedFrom': '441540997',
 'media': [],
 'retweet': True,
 'FavCount': -1,
 'source': 'Twitter for Android',
 'retweet_count': 118}

## Search by a hashtag

In [8]:
# #Find number of tweets with #covid19 (Exact string put the specified string between the ^(Starts with) and $(Ends with))
# myquery = {"hashtags":{"$regex":"^covid19$","$options" :'i'}}
#Search using a hashtag.
myquery = {"content":{"$regex":"#covid19","$options" :'i'}}
twts = list(tweets.find(myquery))
twts[0]

{'_id': ObjectId('5ea66a14fb8f5ff7fd4712a9'),
 'id_str': '1254636770051928071',
 'user_name': 'Pamela Moore #WWG1WGA #DIGITALSOLDIERS',
 'user_id': '1188695886781698050',
 'content': 'Met virtually with our terrific @AmChamPH Board on how U.S. firms are supporting the Philippine economy, workforce, and #COVID19 response. We’re all in this together, and great to \u200bsee U.S. firms contributing in so many ways. #FriendsPartnersAllies #goodbUSinessPH',
 'created_at': '2020-04-27 05:01:30',
 'location': 'Broken Arrow, OK',
 'hashtags': [],
 'mentions': ['441540997', '4745918652'],
 'in_reply_to_user_id': None,
 'in_reply_to_status_id': None,
 'retweetedFrom': '441540997',
 'media': [],
 'retweet': True,
 'FavCount': -1,
 'source': 'Twitter for Android',
 'retweet_count': 118}

In [9]:
len(twts)

397

## Search by time range

In [10]:
# Search by Time range
twts = db.tweets.find({"created_at":{ "$gte": "2020-04-27 05:13:10", "$lt": "2020-04-27 05:13:50" }})
twts = list(twts)
twts[0]

{'_id': ObjectId('5ea66a14fb8f5ff7fd471281'),
 'id_str': '1254639834473230336',
 'user_name': 'Dave Talltree, son of Wymo of Mokuleia',
 'user_id': '1098317762626088961',
 'content': 'Many U.S. businesses unlikely to seek government aid: NABE survey https://t.co/9bPT4YQN5I https://t.co/0B3ekpqymK',
 'created_at': '2020-04-27 05:13:41',
 'location': 'Keiki o ka ʻĀina ',
 'hashtags': [],
 'mentions': ['1652541'],
 'in_reply_to_user_id': None,
 'in_reply_to_status_id': None,
 'retweetedFrom': '1652541',
 'media': ['https://t.co/0B3ekpqymK'],
 'retweet': True,
 'FavCount': -1,
 'source': 'Twitter Web App',
 'retweet_count': 7}

In [11]:
len(twts)

2

## Search by user name

In [12]:
# Search by user_name
twts = db.tweets.find({"user_name":"Howard"})
twts = list(twts)
twts[0]

{'_id': ObjectId('5ea66a14fb8f5ff7fd471282'),
 'id_str': '1254639809068519424',
 'user_name': 'Howard',
 'user_id': '2337724718',
 'content': 'U.S.: bailout small businesses by giving grants to 5% of them\n\nJapan: bailout small businesses by giving 100% of them all the payroll money they need\nhttps://t.co/10BjEDjjn3',
 'created_at': '2020-04-27 05:13:35',
 'location': 'New York, NY',
 'hashtags': [],
 'mentions': ['2172596028'],
 'in_reply_to_user_id': None,
 'in_reply_to_status_id': None,
 'retweetedFrom': '2172596028',
 'media': [],
 'retweet': True,
 'FavCount': -1,
 'source': 'Twitter for iPhone',
 'retweet_count': 240}

In [13]:
len(twts)

1

## Search by user_id

In [14]:
# Search by user id
twts = db.tweets.find({"user_id":"123628682"})
twts = list(twts)
twts[0]

{'_id': ObjectId('5ea66a14fb8f5ff7fd471280'),
 'id_str': '1254639872641576960',
 'user_name': 'Xcntrik',
 'user_id': '123628682',
 'content': 'Many U.S. businesses unlikely to seek government aid: NABE survey https://t.co/9bPT4YQN5I https://t.co/0B3ekpqymK',
 'created_at': '2020-04-27 05:13:50',
 'location': 'Southeast Texas',
 'hashtags': [],
 'mentions': ['1652541'],
 'in_reply_to_user_id': None,
 'in_reply_to_status_id': None,
 'retweetedFrom': '1652541',
 'media': ['https://t.co/0B3ekpqymK'],
 'retweet': True,
 'FavCount': -1,
 'source': 'Twitter for Android',
 'retweet_count': 7}

In [15]:
len(twts)

2

## Search by two words

In [16]:
#search by two words
twts = db.tweets.find( {
    "$and" : [
        {"content":{"$regex":"covid19","$options" :'i'}} , {"content":{"$regex":"#trump","$options" :'i'}}
    ]
} )
twts = list(twts)
twts[0]

{'_id': ObjectId('5ea66a1ffb8f5ff7fd47154e'),
 'id_str': '1254579387464212487',
 'user_name': 'J Cassidy',
 'user_id': '718527880284610561',
 'content': "@realDonaldTrump Read the Constitution. You're embarrassing yourself again... #Trump #coronavirus #COVID19  https://t.co/bCRHRusB3I",
 'created_at': '2020-04-27 01:13:29',
 'location': '',
 'hashtags': ['Trump', 'coronavirus', 'COVID19'],
 'mentions': ['123281100', '25073877'],
 'in_reply_to_user_id': None,
 'in_reply_to_status_id': None,
 'retweetedFrom': '123281100',
 'media': [],
 'retweet': True,
 'FavCount': -1,
 'source': 'Twitter Web App',
 'retweet_count': 826}

In [17]:
len(twts)

9

## Top five hashtags used

In [18]:
hashtag_count = {}
for tweet in db.tweets.find({"hashtags":{"$ne":[]}},{"hashtags":1, "_id":0}):
    ht = tweet["hashtags"]
    for hash in ht:
        hash = hash.lower()
        if hash not in hashtag_count:
            hashtag_count[hash] = 1
        else:
            hashtag_count[hash] += 1
hashtags = pd.DataFrame.from_dict(hashtag_count, orient='index')
hashtags = hashtags.rename(columns={0:"count"})
hashtags = hashtags.sort_values("count", ascending=False)
hashtags.head(5)

,count
covid19,222
trump2020,213
2a41a,209
veterans,209
alwayssupportourveterans,209


## Misc

In [19]:
#???????
myquery = {"location":"New York, NY"}

tweets.count_documents(myquery)

104

In [20]:
#Types of tweets
count_retweet = tweets.count_documents({"retweet":True})
count_original = tweets.count_documents({"retweet":False})
"Number of original tweet:" + str(count_original) + " and number of retweets:" + str(count_retweet)

'Number of original tweet:3306 and number of retweets:12123'

In [21]:
hashtag_count = {}
for tweet in tweets.find():
    ht = tweet["hashtags"]
    for hash in ht:
        hash = hash.lower()
        if hash not in hashtag_count:
            hashtag_count[hash] = 1
        else:
            hashtag_count[hash] += 1

hc = sorted(hashtag_count, key=hashtag_count.get, reverse=True)  # Sorting the top 10.
count = 0

for z in hc:
    if count == 10:
        break
    jsonx = {"hashtag": z, "count": str(hashtag_count[z])}
    count += 1
# {k: v for k, v in sorted(hashtag_count.items(), key=lambda item: item[1])}

## Experimenting with text index

In [28]:
#Connect to the mongoDB database
client = pymongo.MongoClient()
db1 = client["TWT_DB"]
test = db1["test"]

In [29]:
#Create a test index
db1.test.create_index([("content",pymongo.TEXT)])

'content_text'

In [30]:
twts1 = db1.test.find({"$text": {"$search": "\"#covid19\"" } } )
twts1 = list(twts1)
twts1[0]

{'_id': ObjectId('5ea66afbfb8f5ff7fd474756'),
 'id_str': '1253752842361483265',
 'user_name': 'Dr. Marcell Vollmer #SocialDistancing #StayHome',
 'user_id': '99674560',
 'content': 'The Great #Lockdown of the economy has been completely unprecedented, both in terms of the speed of the shutdown and its impact on jobs.\n\n #coronavirus #COVID19 #health #COVIDー19 #worklife #motivation #FutureofWork #Leadership #WorkLifeBalance #USA\n\n https://t.co/yoEHiyJcxc',
 'created_at': '2020-04-24 18:29:05',
 'location': 'Munich, Bavaria',
 'hashtags': ['Lockdown',
  'coronavirus',
  'COVID19',
  'health',
  'COVIDー19',
  'worklife',
  'motivation',
  'FutureofWork',
  'Leadership',
  'WorkLifeBalance',
  'USA'],
 'mentions': [],
 'in_reply_to_user_id': None,
 'in_reply_to_status_id': None,
 'retweetedFrom': -1,
 'media': [],
 'retweet': False,
 'FavCount': 6,
 'source': 'Twitter for iPad',
 'retweet_count': 4}

In [31]:
len(twts1)

396

In [58]:
twts1 = db1.test.find({"$text": {"$search": "covid19"} } )
twts1 = list(twts1)
twts1[0]

{'_id': ObjectId('5ea66afbfb8f5ff7fd474756'),
 'id_str': '1253752842361483265',
 'user_name': 'Dr. Marcell Vollmer #SocialDistancing #StayHome',
 'user_id': '99674560',
 'content': 'The Great #Lockdown of the economy has been completely unprecedented, both in terms of the speed of the shutdown and its impact on jobs.\n\n #coronavirus #COVID19 #health #COVIDー19 #worklife #motivation #FutureofWork #Leadership #WorkLifeBalance #USA\n\n https://t.co/yoEHiyJcxc',
 'created_at': '2020-04-24 18:29:05',
 'location': 'Munich, Bavaria',
 'hashtags': ['Lockdown',
  'coronavirus',
  'COVID19',
  'health',
  'COVIDー19',
  'worklife',
  'motivation',
  'FutureofWork',
  'Leadership',
  'WorkLifeBalance',
  'USA'],
 'mentions': [],
 'in_reply_to_user_id': None,
 'in_reply_to_status_id': None,
 'retweetedFrom': -1,
 'media': [],
 'retweet': False,
 'FavCount': 6,
 'source': 'Twitter for iPad',
 'retweet_count': 4}

In [56]:
len(twts1)

408

In [44]:
db1.test.find({"$text":{"$search": "covid19"}}).explain()

{'queryPlanner': {'plannerVersion': 1,
  'namespace': 'TWT_DB.test',
  'indexFilterSet': False,
  'parsedQuery': {'$text': {'$search': 'covid19',
    '$language': 'english',
    '$caseSensitive': False,
    '$diacriticSensitive': False}},
  'winningPlan': {'stage': 'TEXT',
   'indexPrefix': {},
   'indexName': 'content_text',
   'parsedTextQuery': {'terms': ['covid19'],
    'negatedTerms': [],
    'phrases': [],
    'negatedPhrases': []},
   'textIndexVersion': 3,
   'inputStage': {'stage': 'TEXT_MATCH',
    'inputStage': {'stage': 'FETCH',
     'inputStage': {'stage': 'OR',
      'inputStage': {'stage': 'IXSCAN',
       'keyPattern': {'_fts': 'text', '_ftsx': 1},
       'indexName': 'content_text',
       'isMultiKey': True,
       'isUnique': False,
       'isSparse': False,
       'isPartial': False,
       'indexVersion': 2,
       'direction': 'backward',
       'indexBounds': {}}}}}},
  'rejectedPlans': []},
 'executionStats': {'executionSuccess': True,
  'nReturned': 408,
  'exe

In [36]:
#Connect to the mongoDB database
client = pymongo.MongoClient()
db = client["TWT_DB"]
tweets = db["tweets"]

In [50]:
db.tweets.find({"content":{"$regex":"covid19","$options" :'i'}}).explain()

{'queryPlanner': {'plannerVersion': 1,
  'namespace': 'TWT_DB.tweets',
  'indexFilterSet': False,
  'parsedQuery': {'content': {'$regex': 'covid19', '$options': 'i'}},
  'winningPlan': {'stage': 'COLLSCAN',
   'filter': {'content': {'$regex': 'covid19', '$options': 'i'}},
   'direction': 'forward'},
  'rejectedPlans': []},
 'executionStats': {'executionSuccess': True,
  'nReturned': 413,
  'executionTimeMillis': 11,
  'totalKeysExamined': 0,
  'totalDocsExamined': 15429,
  'executionStages': {'stage': 'COLLSCAN',
   'filter': {'content': {'$regex': 'covid19', '$options': 'i'}},
   'nReturned': 413,
   'executionTimeMillisEstimate': 0,
   'works': 15431,
   'advanced': 413,
   'needTime': 15017,
   'needYield': 0,
   'saveState': 120,
   'restoreState': 120,
   'isEOF': 1,
   'direction': 'forward',
   'docsExamined': 15429},
  'allPlansExecution': []},
 'serverInfo': {'host': 'DESKTOP-T0J0HGC',
  'port': 27017,
  'version': '4.2.6',
  'gitVersion': '20364840b8f1af16917e4c23c1b5f5efd8b3

In [59]:
client.close()